In [24]:
import numpy as np
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV

In [14]:
transform = transforms.Compose([transforms.RandomHorizontalFlip(),transforms.RandomRotation(60),transforms.ToTensor(),transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
total_size = len(train_dataset)
train_size = int(0.8 * total_size)
val_size = total_size - train_size

# Split the dataset into training and validation set
train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size], generator=torch.Generator().manual_seed(42))

# data loaders
train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=False)

Files already downloaded and verified
Files already downloaded and verified


In [15]:
def flatten_images(data_loader):
    flat_images = []
    labels = []
    for images, labels_batch in data_loader:
        flat_images.append(images.view(images.size(0), -1).numpy())
        labels.append(labels_batch.numpy())
    return np.vstack(flat_images), np.hstack(labels)


In [16]:
def apply_pca(data, num_components):
    pca = PCA(n_components=num_components)
    return pca.fit_transform(data)

In [17]:
def evaluate_linear_svm(model, x_val_pca, y_val):
    y_pred_linear = model.predict(x_val_pca)
    print("Linear SVM Classification Report:")
    print(classification_report(y_val, y_pred_linear))

In [18]:
def evaluate_rbf_svm(model, x_val_pca, y_val):
    y_pred_rbf = model.predict(x_val_pca)
    print("RBF SVM Classification Report:")
    print(classification_report(y_val, y_pred_rbf))

In [19]:
def visualize_images(true_labels, pred_labels, images, title):
    correct_indices = np.where(true_labels == pred_labels)[0]
    incorrect_indices = np.where(true_labels != pred_labels)[0]

    plt.figure(figsize=(12, 6))

    # Plot Correctly Classified Images
    plt.subplot(1, 2, 1)
    plt.title('Correctly Classified')
    for i in range(5):
        plt.imshow(images[correct_indices[i]].reshape(32, 32, 3).transpose(1, 2, 0))  # Reshape for image display
        plt.axis('off')
        plt.show()

    # Plot Incorrectly Classified Images
    plt.subplot(1, 2, 2)
    plt.title('Incorrectly Classified')
    for i in range(5):
        plt.imshow(images[incorrect_indices[i]].reshape(32, 32, 3).transpose(1, 2, 0))
        plt.axis('off')
        plt.show()

    plt.suptitle(title)
    plt.show()

In [21]:
x_train_flat, y_train = flatten_images(train_loader)
x_val_flat, y_val = flatten_images(val_loader)
x_test_flat, y_test = flatten_images(test_loader)

In [22]:
x_train_pca = apply_pca(x_train_flat, 66)
x_val_pca = apply_pca(x_val_flat, 66)
x_test_pca = apply_pca(x_test_flat, 66)

In [ ]:
# Train and evaluate RBF SVM by choosing the best possible parameters 
rbf_svm = SVC(kernel='rbf')
param_grid = {'C': [1,10,100], 'gamma': [0.0001,0.001]}
grid_search = GridSearchCV(rbf_svm, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train_pca, y_train)
best_rbfsvm_model = grid_search.best_estimator_
print("Tuned Parameters: ", grid_search.best_params_)
evaluate_rbf_svm(best_rbfsvm_model, x_test_pca, y_test)

In [ ]:
# Train and evaluate linear SVM by choosing the best possible parameters
lin_svm = SVC(kernel='linear')
param_grid1 = {'C': [1,10,100]}
grid_search1 = GridSearchCV(lin_svm, param_grid1, cv=5, scoring='neg_mean_squared_error')
grid_search1.fit(x_train_pca, y_train)
best_linsvm_model = grid_search1.best_estimator_
print("Tuned Parameters: ", grid_search1.best_params_)
evaluate_linear_svm(best_linsvm_model, x_test_pca, y_test)

In [ ]:
# Visualize Correctly and Incorrectly Classified Images
visualize_images(y_val, best_linsvm_model.predict(x_val_pca), x_val, 'Linear SVM')
visualize_images(y_val, best_rbfsvm_model.predict(x_val_pca), x_val, 'RBF SVM')